<a href="https://colab.research.google.com/github/phoenixpereira/GC-Project-1/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.5 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from datetime import timedelta
import os

# Read CSV file
filtered_path = 'filteredDatasets/australia.csv'
data = pd.read_csv(filtered_path)
headlines_data = []
headlines_data.append(data)

headlines_df = pd.concat(headlines_data, ignore_index=True)

# Using a specific model for sentiment analysis
specific_model = pipeline(
    model="siebert/sentiment-roberta-large-english", device=0)  # Use device mps for M1 GPU

total_headlines = len(headlines_df)

sentiments = defaultdict(lambda: defaultdict(int))

# Add tqdm progress bar
with tqdm(total=total_headlines, desc="Analysing Sentiments", unit="headline", dynamic_ncols=True) as pbar:
    for idx in range(total_headlines):
        row = headlines_df.iloc[idx]
        headline = row['headline_text']
        sentiment_date = pd.to_datetime(row['publish_date'], format='%Y-%m-%d')
        sentiment_month = sentiment_date.replace(
            day=1, hour=0, minute=0, second=0, microsecond=0)
        result = specific_model(headline)
        label = result[0]['label']

        if label == 'POSITIVE':
            sentiments[sentiment_month]['POSITIVE'] += 1
        elif label == 'NEGATIVE':
            sentiments[sentiment_month]['NEGATIVE'] += 1

        pbar.update(1)

# Convert the data into a format suitable for plotting
combined_data = {
    'interval': [],
    'publish_date': [],
    'Positive': [],
    'Negative': []
}

for sentiment_month, sentiment_counts in sentiments.items():
    combined_data['interval'].append(sentiment_month)
    combined_data['publish_date'].append(sentiment_month)
    combined_data['Positive'].append(sentiment_counts['POSITIVE'])
    combined_data['Negative'].append(sentiment_counts['NEGATIVE'])

combined_data = pd.DataFrame(combined_data)
combined_data['interval'] = pd.to_datetime(combined_data['interval'])
combined_data['interval'] = combined_data['interval'].dt.to_period('Q')

# Group and aggregate by 3-month intervals
grouped_data = combined_data.groupby('interval', as_index=False).agg({
    'publish_date': 'min',
    'Positive': 'sum',
    'Negative': 'sum'
})

# Save the combined sentiment data to CSV
grouped_data.to_csv('sentimentsAU.csv', index=False)


Analysing Sentiments:   0%|          | 10/54686 [00:01<1:04:12, 14.19headline/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Analysing Sentiments:  11%|█         | 5937/54686 [03:01<24:54, 32.62headline/s]


KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')